# `nwpy` tutorial

Milos Atz 

Last modified on 2018-11-27

In [1]:
%matplotlib inline

## 1. Introduction

The `nwpy` package mimics the mass flow operations described in the Department of Energy Fuel Cycle Evaluation and Screening (FCES) study and produces values for the mass and composition of different waste streams. The package then extends beyond the FCES work by enabling the loading of waste streams into forms suitable for final disposal. These forms can then be used as source term for subsequent waste management calculations, such as repository performance assessment.

## 2. Organization and data

In keeping with the hierarchical fuel cycle organization scheme presented in the FCES report, users of `nwpy.fuelcycle` instantiate a `FuelCycle`, which in turn is made up of `Stages`. The `Stage` is the main object in `fuelcycle` because each fuel cycle stage produces unique, independent waste streams. Upon instantiation, `Stage` objects read the following essential data, along with other descriptive details, that were obtained from the FCES reports and online database. 

1. the mass of used nuclear fuel (UNF) discharged from the irradiation system
2. the cooling time required for UNF after discharge
3. the method used to reprocess UNF, if applicable
4. the species recovered from the UNF, if applicable.

The above data describe the operations that take place within the `Stage`, which is made up of sub-objects that carry them out. The structure of the `Stage` is shown below. The arrangement of operations is similar to that of an FCES stage. 

![alt text](./img/stage.png "Layout of the Stage object")

Each operation acts on a material `Stream` object, which contains characteristic data such as mass and composition, as well as keywords identifying the form of the stream. When streams are created within the `Stage` object, they are loaded with composition data from CSV files associated with the `Stage`. After streams are created, decay calculations, separations, and waste form loading can be applied. This notebook demonstrates the application of some of the methods available in the `fuelcycle` package.

## 3. Fundamental `Stage` methods

The steps below outline the process by which one can calculate waste properties from a `Stage`. These methods are also encapsulated in other `Stage` methods. The ellipses indicate optional keyword arguments that can be passed into the method to alter its behavior. More details about those can be found in the documentation.

| Step | Description                    | Example                                 |
|:-----|:-------------------------------|:----------------------------------------|
| 0.   | Import `nwpy`                  | `from nwpy.fuelcycle import stage`      |
| 1.   | Instantiate the `Stage`        | `s = stage.Stage('egXX', NSTG)`         |
| 2.   | Discharge `Streams`            | `strm = s.discharge_streams()`          |
| 3.   | Cool `Streams`                 | `strm = s.cool(strm, ...) `             |
| 4.   | Reprocess `Streams`            | `w = s.reprocess(strm, ...)`            |
| 5.   | Load `Streams` in `WasteForms` | `wf = s.load_waste(w, ...)`             |
| 6.   | Calculate long-term properties | `wf = s.decay(w, endtime, nsteps, ...)` |

This sequence of methods produces one or more `WasteForm` objects, which are identical to the `Stream` object but with extra attributes: the dimensions of the waste package and the number of waste packages of its kind produced in the stage. At this point, loading is implemented for only major waste streams—those containing the bulk of the decay heat and radioactivity. Support for others may be developed in the future.

**Step 0.** Import the `stage` module from `nwpy`

In [16]:
!pip install -t ./ ../

Processing /Users/harukowainwright/Documents/Projects/Chloe/Codes/nwpy_code
  Preparing metadata (setup.py) ... done
  Created wheel for nwpy: filename=nwpy-0.0.3-py3-none-any.whl size=637651 sha256=00000f4d519eedc8904750ea28be52bbae8a78311b9e44a2d161c0fe87ccf7f0
  Stored in directory: /private/var/folders/53/mywvp84j3zv5w72jr7dh8jth0000gn/T/pip-ephem-wheel-cache-c5gfiinx/wheels/de/1a/a1/2a79d019f1ffbfd489d70753db5166c6a80a71ab9b21fed7b4
Successfully built nwpy


In [17]:
import nwpy

/Users/harukowainwright/Documents/Projects/Chloe/Codes/nwpy_code/notebooks/nwpy/origen.py:212: SyntaxWarning: invalid escape sequence '\.'
  dat=dat.loc[:,~dat.columns.str.replace("(\.\d+)$","").duplicated()]


ModuleNotFoundError: No module named 'imp'

In [4]:
from nwpy import stage

ModuleNotFoundError: No module named 'nwpy'

**Step 1.** Instantiate the `Stage`

In [ ]:
# Example: EG13-1
s = stage.Stage('eg13', 1)

In [ ]:
print s

**Step 2.** Discharge `Streams` from the reactor

In [ ]:
strm = s.discharge_streams()

In [ ]:
print strm

In [ ]:
dir(strm)

In [ ]:
strm.mass_fraction('u238')

**Step 3.** Cool Streams after discharge, before reprocessing/loading

NOTE: Optional keyword argument for `cool`: `rerun=False`. Use if you've already run this case and are just reloading the data

In [ ]:
strm = s.cool(strm)#, rerun=False)

**Step 4.** Reprocess Streams

If the `Stage` does not involve reprocessing, this will return the `Stream`(`s`) unaltered


In [ ]:
w = s.reprocess(strm)

In [ ]:
w

In [ ]:
glass = w[0]
#glass.comp

**Step 5.** Load the waste Streams into WasteForms

If the loading for a certain `Stream` into a `WasteForm` is not supported, this method will skip it

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
wf = s.load_waste(w, plot=True)
plt.savefig('glass_loading.png')

In [ ]:
print(wf)

In [ ]:
wf = s.load_waste(glass)

In [ ]:
wf.canister

**Step 6.** Calculate long-term time-dependent characteristics

In [ ]:
wf = s.decay(wf, endtime=1e5, steps=20)

In [ ]:
wf.tox

These methods are incorporated into broader methods in the `Stage`. After instantiation as `s`, the user can call `s.discharge_all_wastes(...)` to run steps 2-6 above. The keyword arguments for this method are those for `load_waste` and `decay`.

In [ ]:
# wf = s.discharge_all_wastes()

## 2. Plotting

`nwpy.fuelcycle` offers a plotting module to make simple plots of `Stream` data. The module is built on top of the `matplotlib.pyplot` package. Two types of plots are supported: 

1. Pie plots, which can be used to examine the breakdown of a property for a single `Stream`.
2. Time-dependent line plots, which can be used to compare properties of different `Streams`;

The use of those plots is demonstrated here for the `WasteForm` instance we generated above.

### 2.1 Pie plot

The pie plot can be used to compare `Stream` data among isotopes. The user specifies the following inputs:

1. The `Stream` instance
2. The data to be plotted (in this example, the plot will be of decay heat)
3. (optional) If time-dependent data has been generated, the time snapshot at which to plot the data
4. (optional) `maxslices` (for the maximum number of slices in the pie)
5. (optional) `cutoff` (minimum value below which values are not included in the plot). 

In [ ]:
from nwpy.plot import PiePlot as pp

In [ ]:
pp.plot(wf, 'heat', time=100.0, maxslices=5)
pp.label()
# pp.legend() # not yet implemented
pp.show()

### 2.2 Time plot

The time plot can be used to compare time-dependent `Stream` data. The plot can show different elements, isotopes, and species from different `Streams`. The `Stream` total data is plotted using the main method, `plot`, which has the following inputs:

1. The `Stream` instance
2. The data to be plotted (in this example, the plot will be of decay heat)

Optional keyword formatting arguments can be passed to `matplotlib.pyplot`, including:
* color: str indicating color of line and markers
* linestyle: str indicating the style of line (dash vs. solid, etc.)
* marker: str indicating marker style
* markevery: int indicating interval to skip marking data points (good to use for log-plots)

Calling `plot` multiple times with new `Streams` will add all of them to the same plot. To add curves for specific lines, the user can use the `add_element`, `add_isotope`, and `add_species` methods, which have the same inputs as the main `plot` method.

Other commands allow the user to format the axes or add a legend.

In [ ]:
from nwpy.fuelcycle.plot import TimePlot as tp

In [ ]:
tp.plot(wf, 'heat', color='k')
tp.add_element(wf, 'am', linestyle='--', color='r')
tp.add_isotope(wf, 'am241', marker='o', color='r', markevery=2)
tp.set_log('x', 'y')
tp.ylim(ymin=1)
tp.legend()
tp.show()

## 3. `FuelCycle` and benchmarking

The `FuelCycle` object is a collection of `Stages`. It contains methods that can be applied to all the stages in the fuel cycle. One important method is `benchmark`. After separations have been applied, the metric data used to evaluate the waste management criterion in the FCES study can be evaluated. By calculating time-dependent data for the `Streams` produced after reprocessing and summing the results, we can compare fuelcycle results with those from the FCES study. The `benchmark` method prints the ratio of the `nwpy`-calculated result against that of the FCES metric result for 3 of the waste management metrics analyzed in the study

In [ ]:
from nwpy import fuelcycle

In [ ]:
fc = fuelcycle.FuelCycle('eg05')
fc.benchmark()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from nwpy.plot import TimePlot as tp

In [ ]:
s = stage.Stage('eg01', 1)
strm = s.discharge_streams()
w = s.load_waste(strm, loading=1)
w = s.decay(w, endtime=1e6, steps=100)

In [ ]:
tp.plot(w, 'heat', color='k', linewidth=2, label='Total')
tp.add_species(w, 'actinide', linestyle='--', linewidth=2, color='b', label='Actinide')
tp.add_species(w, 'fp', linestyle='--', linewidth=2, color='r', label='FP')#, markevery=2)
tp.set_log('x', 'y')
tp.ylim([0.1, 1e4])
tp.legend()
plt.savefig('./heat.png', dpi=200)
tp.show()